In [21]:
import os
from typing import TypedDict, List, Optional
from pydantic import BaseModel, Field, AliasChoices
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from constants import BASE_URL, API_KEY, MODEL_NAME

In [22]:
# Схемы данных (Pydantic)

class Topic(BaseModel):
    title: str = Field(description="Заголовок темы")
    # Делаем тег необязательным. Если его нет, будет пустая строка или default.
    tag: Optional[str] = Field(
        default="#general", 
        description="Тематический тег, например #ai / #жизньвКитае"
    )
    # Позволяем модели называть preview как угодно
    preview: str = Field(
        validation_alias=AliasChoices('preview', 'description', 'content_summary', 'text'),
        description="Краткое описание"
    )

class TopicProposal(BaseModel):
    # Позволяем модели называть список тем как угодно
    topics: List[Topic] = Field(
        validation_alias=AliasChoices('topics', 'themes', 'ideas', 'items'),
        description="Список уникальных тем"
    )

class DuplicateCheck(BaseModel):
    is_duplicate: bool = Field(description="Флаг дубликата")
    reason: str = Field(
        validation_alias=AliasChoices('reason', 'explanation', 'verdict', 'analysis'),
        description="Пояснение"
    )

# Состояние графа (State)

class AgentState(TypedDict):
    website_context: str
    channel_archive: str
    objectives: str
    style_guide: str
    suggested_topics: List[Topic]
    selected_topics: List[str]
    final_posts: List[str]
    skip_list: List[str]

# Инициализация LLM
llm = ChatOpenAI(
    base_url=BASE_URL, 
    api_key=API_KEY, 
    model=MODEL_NAME,
    temperature=0.7
)


In [23]:
# Агенты

def librarian_agent(state: AgentState):
    """Собирает контекст из файлов."""
    def read_file(path, default):
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as f:
                return f.read()
        return default

    return {
        "website_context": read_file('website_content.md', "No website info available."),
        "channel_archive": read_file('posts.md', "No archive available."),
        "objectives": read_file('objectives.md', "No specific objectives provided.")
    }

def stylist_agent(state: AgentState):
    """Анализирует стиль письма."""
    if "stylist" in state.get("skip_list", []):
        return {"style_guide": "Стандартный информационный стиль."}
    
    prompt = f"Проанализируй стиль этих постов и выдели 5-7 ключевых фишек письма / особенностей написания текстов:\n\n{state['channel_archive']}"
    response = llm.invoke(prompt)
    print(f'Стиль написания: {response.content}')
    return {"style_guide": response.content}

def planner_agent(state: AgentState):
    """Ищет 'пробелы' и предлагает темы."""
    planner_llm = llm.with_structured_output(TopicProposal, method="json_mode")
    
    system_msg = SystemMessage(content=(
        "Ты — контент-стратег. Твоя задача — проанализировать предыдущие посты автора, и на основе этого оопределить следующие темы для написания постов.\n"
        f"ЦЕЛИ: {state['objectives']}\n"
        "Найди темы, которые еще НЕ раскрыты или могут дополнить существующий архив.\n"
        "СТРОГО СОБЛЮДАЙ СТРУКТУРУ JSON: каждое событие в 'topics' "
        "ОБЯЗАТЕЛЬНО должно содержать ключи: 'title', 'tag' и 'preview'." # Явное напоминание
    ))
    
    user_msg = HumanMessage(content=(
        f"Информация о моих предыдущих постах: {state['channel_archive']}\n"
        "Предложи 5-7 уникальных тем."
    ))

    result = planner_llm.invoke([system_msg, user_msg])
    return {"suggested_topics": result.topics}

def gatekeeper_agent(state: AgentState):
    """Проверка на дубликаты."""
    check_llm = llm.with_structured_output(DuplicateCheck, method="json_mode")
    approved = []
    
    for topic in state["suggested_topics"]:
        topic_str = f"{topic.title}: {topic.preview}"
        
        system_msg = SystemMessage(content="Ты шеф-редактор. Сравнивай новую идею с архивом на предмет ИДЕНТИЧНОСТИ советов и выводов.")
        user_msg = HumanMessage(content=(
            f"АРХИВ:\n{state['channel_archive']}\n\n"
            f"НОВАЯ ИДЕЯ:\n{topic_str}\n"
            "Это скучный дубликат? Ответь в JSON."
        ))
        
        res = check_llm.invoke([system_msg, user_msg])
        if not res.is_duplicate:
            approved.append(f"{topic_str} (Контекст: {res.reason})")
            
    return {"selected_topics": approved}

def writer_agent(state: AgentState):
    """Генерация постов (k штук)."""
    if "writer" in state.get("skip_list", []):
        return {"final_posts": ["Writing skipped."]}
    
    generated_posts = []
    # Пишем до 3 постов из одобренных тем
    for topic_context in state["selected_topics"][:3]:
        prompt = f"""
        Напиши пост для Telegram.
        ЦЕЛЬ АВТОРА: {state['objectives']}
        ТЕМА: {topic_context}
        СТИЛЬ: {state['style_guide']}
        КОНТЕКСТ АВТОРА: {state['website_context']}
        ИНФОРМАЦИЯ О ПРЕДЫДУЩИХ ПОСТАХ АВТОРА (ВКЛЮЧАЯ ИХ ТЕКСТ): {state['channel_archive']}

        При написании текста в первую очередь ориентируйся на стиль автора и его предыдущие посты. Твоя задача — создать уникальный и интересный контент, который соответствует видению и стилю автора.
        Пост должен быть информативным, полезным и соответствовать интересам целевой аудитории автора. За счет этого пост будет выглядеть органично в общем контексте канала автора, и будет дополнять предыдущие посты, чтобы читатели узнали что-то новое
        """
        response = llm.invoke(prompt)
        generated_posts.append(response.content)
        
    return {"final_posts": generated_posts}

# Сборка графа

workflow = StateGraph(AgentState)

workflow.add_node("librarian", librarian_agent)
workflow.add_node("stylist", stylist_agent)
workflow.add_node("planner", planner_agent)
workflow.add_node("gatekeeper", gatekeeper_agent)
workflow.add_node("writer", writer_agent)

workflow.set_entry_point("librarian")
workflow.add_edge("librarian", "stylist")
workflow.add_edge("stylist", "planner")
workflow.add_edge("planner", "gatekeeper")
workflow.add_edge("gatekeeper", "writer")
workflow.add_edge("writer", END)

app = workflow.compile()

initial_input = {"skip_list": []} 
try:
    result = app.invoke(initial_input)
    
    print(f"\n✅ Пайплайн завершен. Найдено тем: {len(result['selected_topics'])}")
    print(f"✅ Сгенерировано постов: {len(result['final_posts'])}\n")

    for idx, post in enumerate(result['final_posts'], 1):
        print(f"--- ВАРИАНТ №{idx} ---")
        print(post)
        print("\n" + "="*50 + "\n")
        
except Exception as e:
    print(f"❌ Произошла ошибка: {e}")

Стиль написания: Вот **5-7 ключевых фишек стиля** этих постов, которые делают их узнаваемыми и эффективными:

### 1. **Разговорный, но структурированный стиль**
   - Тексты написаны **легко и естественно**, как будто автор общается с другом (например, *"При этом что самое интересное..."*, *"Это не хорошо и не плохо, это просто другое)"*).
   - При этом **логика изложения четкая**: часто есть вводная часть, основной блок с примерами и вывод/рефлексия.
   - Используются **разговорные обороты** (*"прикольно", "курс закончился в начале ноября, а финальные результаты стали доступны к просмотру в середине января"*).

### 2. **Сравнения и контрасты**
   - Автор активно **сопоставляет опыт** (Россия vs Китай, ВШЭ vs Цинхуа, классический ML vs LLM).
   - Это создает **контекст для читателя** и делает текст более убедительным.
   - Примеры:
     *"В России в магистратуру зачастую идут люди с целью сменить специальность. В хороших зарубежных магистратурах сильно реже встречаются студенты, которые